In [ ]:
# This cell installs all our tools
# - feedparser: For reading the RSS feed (Retrieval)
# - trafilatura: For scraping the full article text (Augmentation)
# - transformers: For loading our summarization model (Generation)
# - sentencepiece: A helper library that our T5 model needs
# - requests: For sending the final message to Telegram
!pip install feedparser trafilatura transformers sentencepiece requests -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.1 MB/s eta 0:00:00


In [ ]:
# Import all the libraries we just installed
import feedparser
import trafilatura
import requests
import time
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

TELEGRAM_BOT_TOKEN = "8502103948:AAFOt8cACI6wiHt7701lH5dHyhq7ZadG7_8"

TELEGRAM_CHAT_ID = "1115250248"

# --- OUR NEW LIST OF FEEDS ---
RSS_URLS = [
    "https://www.news18.com/rss/india.xml",         # CNN-News18 (India)
    "https://feeds.feedburner.com/NDTV-Latest",    # NDTV (Top Stories)
    "https://www.hindustantimes.com/feeds/rss/latest-news/rssfeed.xml", # Hindustan Times
    "https://www.thehindu.com/news/national/?service=rss" # The Hindu (National)
]

In [ ]:
# Load the T5 model and its "dictionary" (tokenizer)
# This is our "Generation" engine
print("Loading summarization model (t5-small)...")

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

print("Model loaded!")

Loading summarization model (t5-small)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded!


In [ ]:
def scrape_article_text(url):
    """
    BLOCK 3: The Scraper (Augmentation)
    Uses trafilatura to get the clean text from a URL.
    """
    print(f"...Scraping text from: {url}")
    # 1. Download the webpage's HTML
    downloaded = trafilatura.fetch_url(url)

    if downloaded:
        # 2. Extract just the main article text
        return trafilatura.extract(downloaded)
    print("...Failed to scrape text.")
    return None

def summarize_text(text):
    """
    BLOCK 4: The Summarizer (Generation)
    Uses our T5 model to summarize the text.
    """
    if not text:
        return "Could not generate summary (no text)."

    print("...Summarizing text...")
    # T5 models need a "prefix" to tell them what task to do
    prompt = "summarize: " + text

    # 1. Turn the text into numbers (tokens)
    inputs = tokenizer.encode(
        prompt,
        return_tensors="pt",  # 'pt' means PyTorch
        max_length=512,       # Max words to read
        truncation=True       # Cut off the text if it's too long
    )

    # 2. Tell the model to generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=150,       # Max words in the summary
        min_length=40,        # Min words in the summary
        length_penalty=2.0,
        num_beams=4,          # A setting to make the summary higher quality
        early_stopping=True
    )

    # 3. Turn the summary's numbers back into text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print("...Summary complete.")
    return summary

def send_telegram_message(message):
    """
    BLOCK 1: The Messenger (Notify)
    Sends our final message to our chat.
    """
    print("...Sending message to Telegram...")

    # This is the simple API URL to send a message
    # We use Markdown for basic formatting (like *bold*)
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

    # This is the "payload" or the "letter" we're sending
    payload = {
        'chat_id': TELEGRAM_CHAT_ID,
        'text': message,
        'parse_mode': 'Markdown' # Allows us to use *bold* and _italic_
    }

    try:
        # Send the message!
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("...Message sent successfully!")
        else:
            # Print any errors
            print(f"...Error sending message: {response.json()}")
    except Exception as e:
        print(f"Failed to send Telegram message: {e}")

In [ ]:
# This is our "database" to remember what we've already sent.
# A 'set' is a fast way to check if we've seen an item.
seen_articles = set()

def run_pipeline():
    print(f"\n--- {time.ctime()} ---")

    # --- NEW: Loop through each URL in our list ---
    for url in RSS_URLS:
        print(f"Checking RSS feed: {url}")

        # BLOCK 2: The RSS Reader (Retrieval)
        # Get all current articles from THIS feed
        feed = feedparser.parse(url)

        new_articles_found = 0

        # We loop in reverse to get the oldest *new* article first
        for entry in reversed(feed.entries):
            article_link = entry.link

            # --- The "Filter" Step ---
            # Check if we've seen this article link before
            if article_link not in seen_articles:
                new_articles_found += 1
                print(f"\nNEW Article Found: {entry.title}")

                # --- RAG PIPELINE ---
                # 1. AUGMENTATION: Scrape the full text
                full_text = scrape_article_text(article_link)

                if full_text:
                    # 2. GENERATION: Summarize the full text
                    summary = summarize_text(full_text)

                    # 3. NOTIFY: Format and send the message
                    message_to_send = f"""
🆕 *New Article Summary*

*Source:* {feed.feed.title}
*Title:* {entry.title}

*Summary:*
{summary}

*Link:* {article_link}
                    """
                    send_telegram_message(message_to_send)
                else:
                    print("...Skipping article, couldn't get text.")

                # --- REMEMBER IT ---
                # Add to our "database" so we don't send it again
                seen_articles.add(article_link)

                # Let's just do one at a time *per feed* to not spam
                break

        if new_articles_found == 0:
            print("No new articles found for this feed.")

# --- The Loop ---
# This will run the pipeline forever.
try:
    # Run it once right now
    run_pipeline()

    while True:
        # Wait for 30 minutes (1800 seconds)
        print(f"\n--- Waiting for 30 minutes... (Press Ctrl+C to stop) ---")
        time.sleep(1800)
        run_pipeline()

except KeyboardInterrupt:
    # This lets us stop the script by pressing the "Stop" button
    print("\nPipeline stopped by user. Goodbye!")


--- Thu Nov 13 17:47:30 2025 ---
Checking RSS feed: https://www.news18.com/rss/india.xml

NEW Article Found: Red Fort Blast & India's ‘White Collar’ Terror Network: The World Wide Web Of ‘Educated Extremists’
...Scraping text from: https://www.news18.com/india/red-fort-blast-indias-white-collar-terror-network-the-world-wide-web-of-educated-extremists-ws-l-9699663.html
...Summarizing text...
...Summary complete.
...Sending message to Telegram...
...Message sent successfully!
Checking RSS feed: https://feeds.feedburner.com/NDTV-Latest
No new articles found for this feed.
Checking RSS feed: https://www.hindustantimes.com/feeds/rss/latest-news/rssfeed.xml
No new articles found for this feed.
Checking RSS feed: https://www.thehindu.com/news/national/?service=rss

NEW Article Found: Latika Subash to contest in Kottayam municipality
...Scraping text from: https://www.thehindu.com/news/national/kerala/latika-subash-to-contest-in-kottayam-municipality/article70275773.ece
...Summarizing text...

# Deployment

In [8]:
%%writefile requirements.txt
feedparser
trafilatura
transformers
sentencepiece
requests
huggingface_hub
streamlit
torch

Writing requirements.txt


In [5]:
%%writefile app.py
# app.py
# This is our permanent, 24/7 deployment script

import streamlit as st
import feedparser
import trafilatura
import requests
import time
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import threading  # <-- NEW: For running in the background
from huggingface_hub import HfApi, HfFolder, hf_hub_download, upload_file
import os

# ==============================================================================
#  LOAD SECRETS FROM HUGGING FACE (NOT from code)
# ==============================================================================
# This is the secure, professional way to handle tokens
# We will add these in the Hugging Face Space settings
TELEGRAM_BOT_TOKEN = st.secrets.get("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = st.secrets.get("TELEGRAM_CHAT_ID")
HF_TOKEN = st.secrets.get("HF_TOKEN")  # A Hugging Face token with "write" access

# ==============================================================================
#  SETTINGS
# ==============================================================================
RSS_URLS = [
    "https://www.news18.com/rss/india.xml",
    "https://feeds.feedburner.com/NDTV-Latest",
    "https://www.hindustantimes.com/feeds/rss/latest-news/rssfeed.xml",
    "https://www.thehindu.com/news/national/?service=rss"
]

# Settings for our persistent database (on Hugging Face Hub)
# IMPORTANT: You must create a public dataset repo on Hugging Face
# and call it "rag-bot-db" or change this name.
DB_REPO_ID = "YOUR_USERNAME/rag-bot-db"  # <-- CHANGE THIS
DB_FILENAME = "seen_links.txt"

# ==============================================================================
#  LOAD MODEL (This runs only once)
# ==============================================================================
@st.cache_resource
def load_model_and_tokenizer():
    print("Loading summarization model (t5-small)...")
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    print("Model loaded!")
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer()

# ==============================================================================
#  HELPER FUNCTIONS (Scrape, Summarize, Send)
# ==============================================================================
def scrape_article_text(url):
    print(f"...Scraping text from: {url}")
    try:
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            return trafilatura.extract(downloaded)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def summarize_text(text):
    if not text:
        return "Could not generate summary (no text)."
    print("...Summarizing text...")
    prompt = "summarize: " + text
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print("...Summary complete.")
    return summary

def send_telegram_message(message):
    print("...Sending message to Telegram...")
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {'chat_id': TELEGRAM_CHAT_ID, 'text': message, 'parse_mode': 'Markdown'}
    try:
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("...Message sent successfully!")
        else:
            print(f"...Error sending message: {response.json()}")
    except Exception as e:
        print(f"Failed to send Telegram message: {e}")

# ==============================================================================
#  DATABASE FUNCTIONS (Our new "memory")
# ==============================================================================
def get_seen_links():
    """Downloads the list of seen links from our HF Dataset repo."""
    print("...Downloading seen links database...")
    try:
        # Download the file from the Hub
        local_path = hf_hub_download(
            repo_id=DB_REPO_ID,
            filename=DB_FILENAME,
            repo_type="dataset",
            token=HF_TOKEN
        )
        with open(local_path, 'r') as f:
            # Read all lines into a set for fast lookup
            links = set(line.strip() for line in f)
        print(f"...Loaded {len(links)} seen links.")
        return links
    except Exception as e:
        # If the file doesn't exist yet, just return an empty set
        print(f"Warning: Could not download seen links: {e}. Starting fresh.")
        return set()

def update_seen_links(seen_links):
    """Uploads the updated list of links back to the HF Dataset repo."""
    print(f"...Uploading {len(seen_links)} seen links to database...")
    try:
        # Write the set back to a local file
        with open(DB_FILENAME, 'w') as f:
            for link in seen_links:
                f.write(f"{link}\n")

        # Upload the file to the Hub
        upload_file(
            path_or_fileobj=DB_FILENAME,
            path_in_repo=DB_FILENAME,
            repo_id=DB_REPO_ID,
            repo_type="dataset",
            token=HF_TOKEN,
            commit_message="Update seen links"
        )
        print("...Database updated.")
    except Exception as e:
        print(f"Error uploading seen links: {e}")

# ==============================================================================
#  THE MAIN PIPELINE (This will run in the background)
# ==============================================================================
def run_pipeline():
    # 1. Get our "memory" from the database
    seen_articles = get_seen_links()

    print(f"\n--- {time.ctime()} ---")
    print("BOT IS RUNNING. Checking all feeds...")

    new_links_added = False
    for url in RSS_URLS:
        print(f"Checking RSS feed: {url}")
        feed = feedparser.parse(url)

        for entry in reversed(feed.entries):
            article_link = entry.link

            if article_link not in seen_articles:
                print(f"\nNEW Article Found: {entry.title}")
                full_text = scrape_article_text(article_link)

                if full_text:
                    summary = summarize_text(full_text)
                    message_to_send = f"""
🆕 *New Article Summary*
*Source:* {feed.feed.title}
*Title:* {entry.title}
*Summary:*
{summary}
*Link:* {article_link}
                    """
                    send_telegram_message(message_to_send)

                    # Add to our local set
                    seen_articles.add(article_link)
                    new_links_added = True
                else:
                    print("...Skipping article, couldn't get text.")

                # Only process one new article per feed to avoid spamming
                break

    # 2. If we added new links, update our "memory" in the database
    if new_links_added:
        update_seen_links(seen_articles)
    else:
        print("No new articles found this cycle.")

def background_task():
    """This function runs our pipeline in a loop forever."""
    print("Background thread started. Bot is now running.")
    while True:
        try:
            run_pipeline()
            # Wait for 30 minutes
            print(f"\n--- Waiting for 30 minutes... ---")
            time.sleep(1800)
        except Exception as e:
            print(f"Error in main loop: {e}. Restarting in 60s.")
            time.sleep(60)

# ==============================================================================
#  STREAMLIT APP (The "Dummy" UI)
# ==============================================================================
st.title("🤖 RAG Telegram Bot")
st.write("This bot is running 24/7 in the background.")
st.success("The pipeline is active and checking for new articles every 30 minutes.")

st.header("Feeds Being Monitored:")
for url in RSS_URLS:
    st.write(f"- `{url}`")

# --- THIS IS THE MAGIC ---
# Check if the background thread is already running
# We use st.session_state to remember
if "bot_thread" not in st.session_state:
    print("Starting background bot thread...")
    # Create and start the new thread
    thread = threading.Thread(target=background_task, daemon=True)
    thread.start()
    st.session_state.bot_thread = thread

Writing app.py
